In [1]:
import pandas as pd
import geopandas as gpd

import numpy as np

In [2]:
file_names = ["../input/delitos.csv", "../input/tweets-delitos-otro_grupo.csv"]

df = pd.concat([pd.read_csv(name) for name in file_names])

In [3]:
df

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas,Unnamed: 0
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.0,8.0,2314.0,1,"-2.1384511, -79.8913772",NaN
1,Segura EP,@segura_ep,2023-09-01 02:19:12+00:00,Desde las cámaras de videovigilancia captamos ...,0.0,1.0,7.0,2314.0,1,"-2.230788, -79.8939582",NaN
2,Segura EP,@segura_ep,2023-08-31 01:53:39+00:00,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6.0,1335.0,1,"-2.2438513, -79.8956694",NaN
3,Segura EP,@segura_ep,2023-08-29 17:10:33+00:00,Un vehículo reportado como robado fue captado ...,0.0,6.0,20.0,2499.0,1,"-2.1749333, -79.9417115",NaN
4,Segura EP,@segura_ep,2023-08-29 08:04:47+00:00,#Alerta | Miembros de la \r\n@ATM_Transito\r\n...,1.0,5.0,13.0,2499.0,1,"-3.9952065, -79.2022123",NaN
...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,2023-07-12 08:59:00,Un cuerpo desmembrado y embalado fue encontrad...,NaN,NaN,NaN,NaN,1,"-2.1222554, -79.9679769",114.0
303,NaN,NaN,2023-07-13 10:51:00,Delincuentes asaltaron en un local de ropa ubi...,NaN,NaN,NaN,NaN,1,"-2.194905, -79.884937",115.0
304,NaN,NaN,2023-07-13 11:11:00,Delincuente fue neutralizado cuando intentó ro...,NaN,NaN,NaN,NaN,1,"34.0165053, -118.1137535",116.0
305,NaN,NaN,2023-07-13 12:32:00,Delincuentes destrozaron un negocio porque el ...,NaN,NaN,NaN,NaN,1,"-2.187904, -79.881483",117.0


In [4]:
df.dtypes

DisplayName      object
UserName         object
Timestamp        object
Content          object
Comments        float64
Likes           float64
Retweets        float64
Interactions    float64
Delito            int64
Coordenadas      object
Unnamed: 0      float64
dtype: object

In [5]:
def str_to_int_milesimas(value):
    if isinstance(value, str):
        value = value.split('\r')[0]  # Eliminar todo después de '\r'
        value = ''.join(filter(lambda char: char.isdigit() or char.upper() in ['K', 'M'], value))
        if value.endswith('K'):
            value = value[:-1]
            value = int(value) * 1000
        elif value.endswith('M'):
            value = value[:-1]
            value = int(value) * 1000000
        else:
            value = int(value)
        return int(value)
    return value

### Colocamos el formato de los datos

In [6]:
df['Comments'] = df['Comments'].fillna(0)

In [7]:
df['Likes'] = df['Likes'].apply(str_to_int_milesimas)
df['Likes'] = df['Likes'].fillna(0)

In [8]:
df['Retweets'] = df['Retweets'].apply(str_to_int_milesimas)
df['Retweets'] = df['Retweets'].fillna(0)

In [9]:
df['Interactions'] = df['Interactions'].apply(str_to_int_milesimas)
df['Interactions'] = df['Interactions'].fillna(0)

In [10]:
df.dtypes

DisplayName      object
UserName         object
Timestamp        object
Content          object
Comments        float64
Likes           float64
Retweets        float64
Interactions    float64
Delito            int64
Coordenadas      object
Unnamed: 0      float64
dtype: object

In [12]:
df.head()

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas,Unnamed: 0
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.0,8.0,2314.0,1,"-2.1384511, -79.8913772",NaN
1,Segura EP,@segura_ep,2023-09-01 02:19:12+00:00,Desde las cámaras de videovigilancia captamos ...,0.0,1.0,7.0,2314.0,1,"-2.230788, -79.8939582",NaN
2,Segura EP,@segura_ep,2023-08-31 01:53:39+00:00,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6.0,1335.0,1,"-2.2438513, -79.8956694",NaN
3,Segura EP,@segura_ep,2023-08-29 17:10:33+00:00,Un vehículo reportado como robado fue captado ...,0.0,6.0,20.0,2499.0,1,"-2.1749333, -79.9417115",NaN
4,Segura EP,@segura_ep,2023-08-29 08:04:47+00:00,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.0,13.0,2499.0,1,"-3.9952065, -79.2022123",NaN


### Limpiamos y colocamos las coordenadas

In [13]:
df[["latitude", "longitude"]] = df["Coordenadas"].str.split(",", expand=True)

In [14]:
df["latitude"] = df["latitude"].astype(float)
df["longitude"] = df["longitude"].astype(float)

In [15]:
df

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas,Unnamed: 0,latitude,longitude
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.0,8.0,2314.0,1,"-2.1384511, -79.8913772",NaN,-2.138451,-79.891377
1,Segura EP,@segura_ep,2023-09-01 02:19:12+00:00,Desde las cámaras de videovigilancia captamos ...,0.0,1.0,7.0,2314.0,1,"-2.230788, -79.8939582",NaN,-2.230788,-79.893958
2,Segura EP,@segura_ep,2023-08-31 01:53:39+00:00,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6.0,1335.0,1,"-2.2438513, -79.8956694",NaN,-2.243851,-79.895669
3,Segura EP,@segura_ep,2023-08-29 17:10:33+00:00,Un vehículo reportado como robado fue captado ...,0.0,6.0,20.0,2499.0,1,"-2.1749333, -79.9417115",NaN,-2.174933,-79.941711
4,Segura EP,@segura_ep,2023-08-29 08:04:47+00:00,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.0,13.0,2499.0,1,"-3.9952065, -79.2022123",NaN,-3.995207,-79.202212
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,2023-07-12 08:59:00,Un cuerpo desmembrado y embalado fue encontrad...,0.0,0.0,0.0,0.0,1,"-2.1222554, -79.9679769",114.0,-2.122255,-79.967977
303,NaN,NaN,2023-07-13 10:51:00,Delincuentes asaltaron en un local de ropa ubi...,0.0,0.0,0.0,0.0,1,"-2.194905, -79.884937",115.0,-2.194905,-79.884937
304,NaN,NaN,2023-07-13 11:11:00,Delincuente fue neutralizado cuando intentó ro...,0.0,0.0,0.0,0.0,1,"34.0165053, -118.1137535",116.0,34.016505,-118.113754
305,NaN,NaN,2023-07-13 12:32:00,Delincuentes destrozaron un negocio porque el ...,0.0,0.0,0.0,0.0,1,"-2.187904, -79.881483",117.0,-2.187904,-79.881483


### Localizamos geograficamente solo los delitos

In [16]:
df_delitos = df.loc[df["Delito"]==1,:]

In [17]:
df_delitos

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas,Unnamed: 0,latitude,longitude
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.0,8.0,2314.0,1,"-2.1384511, -79.8913772",NaN,-2.138451,-79.891377
1,Segura EP,@segura_ep,2023-09-01 02:19:12+00:00,Desde las cámaras de videovigilancia captamos ...,0.0,1.0,7.0,2314.0,1,"-2.230788, -79.8939582",NaN,-2.230788,-79.893958
2,Segura EP,@segura_ep,2023-08-31 01:53:39+00:00,Desde las cámaras de videovigilancia realizamo...,2.0,3.0,6.0,1335.0,1,"-2.2438513, -79.8956694",NaN,-2.243851,-79.895669
3,Segura EP,@segura_ep,2023-08-29 17:10:33+00:00,Un vehículo reportado como robado fue captado ...,0.0,6.0,20.0,2499.0,1,"-2.1749333, -79.9417115",NaN,-2.174933,-79.941711
4,Segura EP,@segura_ep,2023-08-29 08:04:47+00:00,#Alerta | Miembros de la \n@ATM_Transito\n y \...,1.0,5.0,13.0,2499.0,1,"-3.9952065, -79.2022123",NaN,-3.995207,-79.202212
...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,NaN,NaN,2023-07-12 08:59:00,Un cuerpo desmembrado y embalado fue encontrad...,0.0,0.0,0.0,0.0,1,"-2.1222554, -79.9679769",114.0,-2.122255,-79.967977
303,NaN,NaN,2023-07-13 10:51:00,Delincuentes asaltaron en un local de ropa ubi...,0.0,0.0,0.0,0.0,1,"-2.194905, -79.884937",115.0,-2.194905,-79.884937
304,NaN,NaN,2023-07-13 11:11:00,Delincuente fue neutralizado cuando intentó ro...,0.0,0.0,0.0,0.0,1,"34.0165053, -118.1137535",116.0,34.016505,-118.113754
305,NaN,NaN,2023-07-13 12:32:00,Delincuentes destrozaron un negocio porque el ...,0.0,0.0,0.0,0.0,1,"-2.187904, -79.881483",117.0,-2.187904,-79.881483


In [18]:
# Usamos un diccionario para dividir por zonas a los barrios

ZONAS = {}
ZONAS.update(dict.fromkeys([
        'Guasmo Sur', 
        'Guasmo Norte - Rio Guayas', 
        'Guasmo Oeste', 
        'Floresta',
        'Valdivia',
        'Los Esteros',
        'Sopeña',
        'Pradera',
        'Cuba',
        'Ciudadela 9 de Octubre',
        'Huancavilca Sur',
        'Luz del Guayas',
        'Guanguala',
        'Urbanizacion La Saiba',
        'Acacias',
        'Las Americas',
        'Isla Trinitaria',
        'Centenario',
        'Parroquia Garcia Moreno',
        'Cristo del Consuelo',
        'Puerto Lisa',
        'Batallon del Suburbio',
        'Estero Salado',
        'Abel Gilbert',
        'Barrio Lindo',
        'Parroquia Letamendi',
        'Febres Cordero',
        'Parroquia Ayacucho',
        'Jardines del Salado',
        'Girasol',
        'Renacer',
        'Cooperativa Portete de Tarqui',
        'Bellavista',
        'Puerto Azul',
        'Bosques de la Costa',
        'Portal al Sol',
        'Bello Horizonte',
        'Portofino',
        'Laguna Club',
        'Urbanizacion Portovita',
        'Casa Club',
        'Centro Comercial Via la Costa',
        'Punta Esmeralda',
        'TerraNostra',
        'Arcadia',
        'Academia Logos',
        'Urbanizacion Via al Sol',
        'Puerto Hondo',
        'Luxury',
        'Costalmar 2',
        'Costabrisa',
        'Chongon',
        'Villas del Bosque',
        'Valle Alto'
    ], 'Sur'))

ZONAS.update(dict.fromkeys([
        'Parroquia Urdaneta',
        'Parroquia Sucre',
        'Parroquia Rocafuerte',
        'Parroquia 9 de Octubre',
        'Parroquia Bolivar',
        'Parroquia Olmedo',
        'Parroquia Roca',
        'Parroquia Pedro Carbo',
        'Barrio Orellana',
        'Ciudadela Bolivariana',
        'Universidad de Guayaquil',
        'Cerro del Carmen',
        'Las Peñas',
        'Ciudadela Modelo',
        'Puerto Santa Ana',
    ], 'Centro'))

ZONAS.update(dict.fromkeys([
        'Mapasingue',
        'Ceibos',
        'Urbanor',
        'Urdenor',
        'Bosques del Salado',
        'La Kennedy',
        'Atarazana',
        'Ciudadela La FAE',
        'Urbanizacion Rio Guayas',
        'Urdesa',
        'Miraflores',
        'Ciudad Colon',
        'Martha de Roldos',
        'San Felipe',
        'Prosperina',
        'Santa Beatriz',
        'Juan Montalvo',
        'Colinas de la Alborada',
        'Condor',
        'Alborada',
        'Sauces',
        'Ciudadela Los Alamos',
        'Acuarela',
        'Limonal',
        'Samanes',
        'Rosales',
        'Guayacanes',
        'Florida',
        'Colinas de la Florida',
        'Lotizacion Expogramos',
        'Fuerte Huancavilca',
        'Cerro Colorado',
        'Mucho Lote',
        'Vergeles',
        'Huancavilca Norte',
        'Ciudadela Estrella del Mar',
        'Orquideas',
        'Geranios 1',
        'Geranios 2',
        'Nueva Prosperina',
        'Socio Vivienda',
        'Monte Sinai',
        'Sergio Toral',
        'Balerio Estacio',
        'Flor de Bastion',
        'Ecocity',
        'El Fortin',
        'Urbanizacion Industrial Inmaconsa',
        'Parque Industrial El Sauce',
        'Colinas del Sol',
        'Montebello',
        'Galavsa',
        'Bastion Popular',
        'Ciudadela Caracol',
        'Pascuales',
        'Cooperativa 5 de Diciembre',
        'Lotizacion Industrial Pascuales',
        'Porton de Beata',
        'San Francisco',
        'Metropolis 2',
        'Ciudad del Rio 2',
        'Veranda',
        'La Perla',
        'Romareda',
        'Mucho Lote 2',
        'Jardines del Rio',
        'San Eduardo',
        'Terminal Norte Metrovia',
        'Terminal Terrestre',
        'Garzota',
        'Ciudadela Adace',
        'Ciudadela Vernaza',
        'Ciudadela Simon Bolivar',
        'Brisas del Norte',
        'Ciudad Santiago',
        'Lago Capeira',
        'Villa Bonita'
    ], 'Norte'))

In [19]:
gdf_delitos = gpd.GeoDataFrame(df_delitos, geometry=gpd.points_from_xy(df_delitos["longitude"], df_delitos["latitude"]))

In [20]:
barrios = gpd.read_file("./barrios.geojson")

In [21]:
gdf_delitos.crs = barrios.crs

In [22]:
df_delitos = gdf_delitos.sjoin(barrios, how="inner")

In [23]:
df_delitos["Zona"] = df_delitos.apply(lambda row: ZONAS.get(row["Name"]), axis=1)

In [24]:
df_delitos.dtypes

DisplayName         object
UserName            object
Timestamp           object
Content             object
Comments           float64
Likes              float64
Retweets           float64
Interactions       float64
Delito               int64
Coordenadas         object
Unnamed: 0         float64
latitude           float64
longitude          float64
geometry          geometry
index_right          int64
stroke              object
stroke-width       float64
stroke-opacity     float64
fill                object
fill-opacity       float64
Name                object
Description         object
Zona                object
dtype: object

In [25]:
df_delitos.head()

,DisplayName,UserName,Timestamp,Content,Comments,Likes,Retweets,Interactions,Delito,Coordenadas,...,geometry,index_right,stroke,stroke-width,stroke-opacity,fill,fill-opacity,Name,Description,Zona
0,Segura EP,@segura_ep,2023-09-01 03:20:23+00:00,Desde las cámaras de videovigilancia observamo...,0.0,2.0,8.0,2314.0,1,"-2.1384511, -79.8913772",...,POINT (-79.89138 -2.13845),170,#555555,2.0,1.0,#555555,0.5,Sauces,NaN,Norte
27,Segura EP,@segura_ep,2023-08-24 22:08:45+00:00,#Alerta | Desde las cámaras de videovigilancia...,0.0,3.0,7.0,840.0,1,"-2.1269033, -79.9000788",...,POINT (-79.90008 -2.12690),170,#555555,2.0,1.0,#555555,0.5,Sauces,NaN,Norte
41,Segura EP,@segura_ep,2023-08-22 05:37:57+00:00,Cámaras de videovigilancia monitorean el accid...,0.0,3.0,17.0,2266.0,1,"-2.1329058, -79.8899186",...,POINT (-79.88992 -2.13291),170,#555555,2.0,1.0,#555555,0.5,Sauces,NaN,Norte
107,Segura EP,@segura_ep,2023-07-30 06:50:26+00:00,#Alerta | Desde las cámaras de videovigilancia...,0.0,2.0,18.0,1490.0,1,"-2.1413962, -79.8887857",...,POINT (-79.88879 -2.14140),170,#555555,2.0,1.0,#555555,0.5,Sauces,NaN,Norte
149,Segura EP,@segura_ep,2023-07-22 00:29:11+00:00,Cámaras de videovigilancia captan el accidente...,0.0,1.0,1.0,1240.0,1,"-2.1329058, -79.8899186",...,POINT (-79.88992 -2.13291),170,#555555,2.0,1.0,#555555,0.5,Sauces,NaN,Norte


In [26]:
df_delitos.to_file("tweets-delitos-localizados.geojson", driver="GeoJSON")